# Análisis de datos de minst

In [150]:
import gzip, pickle
import math
import numpy as np
with gzip.open('mnist.pkl.gz','rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    train_set, val_set, test_set = u.load()
print(train_set[0].shape, train_set[1].shape)
print(val_set[0].shape, val_set[1].shape)
print(test_set[0].shape, test_set[1].shape)


(50000, 784) (50000,)
(10000, 784) (10000,)
(10000, 784) (10000,)


In [203]:
lista=list()
for i in range(train_set[0].shape[0]):
    if int(train_set[1][i])==1 or int(train_set[1][i])==0:
        lista.append((int(train_set[1][i]),(train_set[0][i]-np.mean(train_set[0][i]))/np.std(train_set[0][i])))


In [204]:
from numpy import linalg as la
def pi_x(x,i):
    return 1/(1+math.exp(-1*np.dot(lista[i][1].T,x[:-1])-x[-1]))

def log_likelihood(beta):
    suma=0
    for i in range(len(lista)):
        suma += lista[i][0]*math.log(pi_x(beta,i))+(1-lista[i][0])*math.log(1-pi_x(beta,i))
    return suma

In [205]:
def der_pix(x,i):
    k=math.exp(-1*np.dot(lista[i][1].T,x[:-1])-x[-1])
    l=(1+math.exp(-1*np.dot(lista[i][1].T,x[:-1])-x[-1]))**2
    vec=np.zeros(785)
    for j in range(784):
        vec[j]=lista[i][1][j]
    vec[784]=1
    return k/l*vec

def der_likelihood(beta):
    return 1*sum((lista[i][0]/pi_x(beta,i)-((1-lista[i][0])/(1-pi_x(beta,i))))*der_pix(beta,i) for i in range(len(lista)))

In [206]:
def backtracking_direction(x,a,p,c,dd,tf): #Veremos si el alpha satisface la condción de Armijo para el suficiente descenso
    alpha=a
    while tf(x+alpha*dd)> tf(x)+c*alpha*np.dot(dd.T,dd):
        alpha=p*alpha
    return alpha #Este es el alpha óptimo 

def Barzilai_Borwein_descenso(x0,maxitr,epsilon,grad,tf): #la función en este caso no se ocupa
    k=0
    g_k=grad(x0)
    g=-1*g_k
    x_k=x0
    iterations=0
    while la.norm(g_k)>epsilon and iterations<maxitr:
        if k==0:
            a=backtracking_direction(x_k,1,0.5,0.5,g,tf)
        else:
            a=np.dot((x_k-x_kant).T,(g_k-g_k_t))*(1/np.dot((g_k-g_k_t).T,(g_k-g_k_t)))
        x_kant=x_k
        g_k_t=grad(x_kant)
        x_k=x_k+a*g
        g_k=grad(x_k)
        g=-1*g_k
        k=k+1
        iterations=iterations+1
        print(la.norm(g_k))
    return np.array([x_k,g_k])

def modified_backtracking_direction(x,C,a,p,c1,dd,tf): #Veremos si el alpha satisface la condción de Armijo para el suficiente descenso
    alpha=a
    while tf(x+alpha*dd) > C +c1*alpha*np.dot(dd.T,dd):
        alpha=p*alpha
    return alpha #Este es el alpha óptimo 

def Zhang_Hager(x0,maxitr,epsilon,nu,grad,tf):
    k=0
    g_k=grad(x0)
    g=-1*g_k
    iterations=0
    Q=1
    x_k=x0
    C=tf(x_k)
    trial=modified_backtracking_direction(x_k,C,1,0.5,0.5,g,tf)
    while la.norm(g_k)>epsilon and iterations<maxitr:
        a=modified_backtracking_direction(x_k,C,trial,0.5,0.5,g,tf)
        trial=a
        Q_ant=Q
        x_k=x_k+a*g
        g_k=grad(x_k)
        g=-1*g_k
        Q=Q_ant+1
        C=(nu*Q_ant*C+tf(x_k))/Q
        k=k+1
    return np.array([x_k,g_k])

In [210]:
puntoinic=np.zeros(785)
res=der_likelihood(puntoinic)
print(la.norm(res))

7891.244353120374


In [208]:
res=Barzilai_Borwein_descenso(puntoinic,1000,1e-4,der_likelihood,log_likelihood)
print(res)

OverflowError: math range error